In [1]:
import os
import pickle
import json
import numpy as np
from sklearn.metrics import f1_score

In [43]:
obj

{'n_entities': 3540,
 'n_mentions': 5202,
 'knn_mentions': 0,
 'accuracy': '20.722798923490966 %',
 'failure': [{'mention_id': 752023942559326208,
   'mention_name': 'user and exactly what has killary done for our veterans except get them killed in benghazi disgraceful traitor',
   'mention_gold_cui': '393',
   'mention_gold_cui_name': 'PolitiFact Clinton 7 Benghazi probes so far',
   'predicted_name': 'PolitiFact Prior to Benghazi were there 13 attacks on embassies and 60 deaths under President George W Bush',
   'predicted_cui': '272'},
  {'mention_id': 765668466103910400,
   'mention_name': 'person person no terrorist attacks on u s soil before person aside from number thats still false url',
   'mention_gold_cui': '3449',
   'mention_gold_cui_name': 'PolitiFact Aside from 911 Rudy Giuliani is wrong about no terrorist attacks before Obama',
   'predicted_name': 'PolitiFact Prior to Benghazi were there 13 attacks on embassies and 60 deaths under President George W Bush',
   'predicte

In [40]:
modes = ['fulltext', 'summaries']
models = ['in_batch', 'knn', 'mst', '1nn']

for mode in modes:
    print(f"****{mode.upper()}****\n")
    for model in models:
        print(model.upper() + ":")
        print("---")
        fname = f'{mode}_{model}.json'
        fpath = os.path.join(f'data/learnffc/results_{mode}', fname)

        with open(fpath, 'r') as handle:
            obj = json.loads(handle.read())

        y_true = [o['mention_gold_cui'] for o in obj['success']] + [o['mention_gold_cui'] for o in obj['failure']]
        y_pred = [o['predicted_cui'] for o in obj['success']] + [o['predicted_cui'] for o in obj['failure']]

        micro = f1_score(y_true, y_pred, average='micro')
        macro = f1_score(y_true, y_pred, average='macro')
        weighted = f1_score(y_true, y_pred, average='weighted')

        print(f"Micro F1 \t= {micro}")
        print(f"Macro F1 \t= {macro}")
        print(f"Weighted F1 \t= {weighted}\n")
    print("")

****FULLTEXT****

IN_BATCH:
---
Micro F1 	= 0.2522106881968474
Macro F1 	= 0.14514828298030136
Weighted F1 	= 0.3012559743416845

KNN:
---
Micro F1 	= 0.16359092656670512
Macro F1 	= 0.05735379643074781
Weighted F1 	= 0.1510031412787697

MST:
---
Micro F1 	= 0.16782006920415224
Macro F1 	= 0.11760624083819282
Weighted F1 	= 0.1704446785886571

1NN:
---
Micro F1 	= 0.1987697039600154
Macro F1 	= 0.11534326230341112
Weighted F1 	= 0.23624765190297004


****SUMMARIES****

IN_BATCH:
---
Micro F1 	= 0.2518262206843522
Macro F1 	= 0.1319769823108834
Weighted F1 	= 0.29822099493119775

KNN:
---
Micro F1 	= 0.12995001922337562
Macro F1 	= 0.06951155680993117
Weighted F1 	= 0.11970937866405201

MST:
---
Micro F1 	= 0.20626682045367167
Macro F1 	= 0.12597193478646526
Weighted F1 	= 0.21043688626116286

1NN:
---
Micro F1 	= 0.20722798923490965
Macro F1 	= 0.11526179093914941
Weighted F1 	= 0.2524893118388404




In [61]:
# Filter mentions containing ["url", "person", "user"]
def filter_obj(obj, filters=["url", "person", "user"]):
    failure = obj["failure"]
    success = obj["success"]
    f,s = [], []
    for o in failure:
        split = o["mention_name"].split()
        skip = False
        for fil in filters:
            if fil in split:
                skip = True
                break
        if not skip:
            f.append(o)
    for o in success:
        split = o["mention_name"].split()
        skip = False
        for fil in filters:
            if fil in split:
                skip = True
                break
        if not skip:
            s.append(o)
    obj["failure"] = f
    obj["success"] = s

    
modes = ['fulltext', 'summaries']
models = ['in_batch', 'knn', 'mst', '1nn']

for mode in modes:
    print(f"****{mode.upper()}****\n")
    for model in models:
        print(model.upper() + ":")
        print("---")
        fname = f'{mode}_{model}.json'
        fpath = os.path.join(f'data/learnffc/results_{mode}', fname)

        with open(fpath, 'r') as handle:
            obj = json.loads(handle.read())
        
        print(f"Unfiltered length = {len(obj['success'])+len(obj['failure'])}")
        filter_obj(obj)
        print(f"Filtered length = {len(obj['success'])+len(obj['failure'])}")
        
        
        y_true = [o['mention_gold_cui'] for o in obj['success']] + [o['mention_gold_cui'] for o in obj['failure']]
        y_pred = [o['predicted_cui'] for o in obj['success']] + [o['predicted_cui'] for o in obj['failure']]

        micro = f1_score(y_true, y_pred, average='micro')
        macro = f1_score(y_true, y_pred, average='macro')
        weighted = f1_score(y_true, y_pred, average='weighted')

        print(f"Micro F1 \t= {micro}")
        print(f"Macro F1 \t= {macro}")
#         print(f"Weighted F1 \t= {weighted}")
        print("")
    print("")

****FULLTEXT****

IN_BATCH:
---
Unfiltered length = 5202
Filtered length = 277
Micro F1 	= 0.34296028880866425
Macro F1 	= 0.21547960158275342

KNN:
---
Unfiltered length = 5202
Filtered length = 277
Micro F1 	= 0.11191335740072202
Macro F1 	= 0.05642342396294965

MST:
---
Unfiltered length = 5202
Filtered length = 277
Micro F1 	= 0.2851985559566787
Macro F1 	= 0.16668257432588643

1NN:
---
Unfiltered length = 5202
Filtered length = 277
Micro F1 	= 0.2815884476534296
Macro F1 	= 0.17118588482224847


****SUMMARIES****

IN_BATCH:
---
Unfiltered length = 5202
Filtered length = 277
Micro F1 	= 0.3285198555956679
Macro F1 	= 0.19738756613756614

KNN:
---
Unfiltered length = 5202
Filtered length = 277
Micro F1 	= 0.13357400722021662
Macro F1 	= 0.09379058441558441

MST:
---
Unfiltered length = 5202
Filtered length = 277
Micro F1 	= 0.2815884476534296
Macro F1 	= 0.167023199023199

1NN:
---
Unfiltered length = 5202
Filtered length = 277
Micro F1 	= 0.2924187725631769
Macro F1 	= 0.1679287015

True